In [1]:
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

from navec import Navec
from slovnet import NER

In [2]:
path_corpus_tut = Path('../data/corpora/tutby_126784.csv')
path_ner_data = Path('../data/ner/ner_tutby_126784.json')

path_navec = Path('../data/model/natasha/navec_news_v1_1B_250K_300d_100q.tar')
path_slovnet = Path('../data/model/natasha/slovnet_ner_news_v1.tar')

In [3]:
data = pd.read_csv(path_corpus_tut)

corpus = data['document']

print(data.shape)
display(data.head(3))

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,['министерства']


In [4]:
%%time

navec = Navec.load(path_navec)
ner = NER.load(path_slovnet)
ner.navec(navec)


entities = []

for text in tqdm(corpus):
    try:
        markup = ner(text)
        entities += [[(text[span.start:span.stop], span.type) for span in markup.spans]]
    except:
        entities += [[]]
        print(f'--->\n{text}\n<---')
    
entities = [list(dict(entity).items()) for entity in entities]

print(len(entities))




























126784
Wall time: 1h 17min 53s


In [5]:
df = pd.DataFrame()

df['url'] = data['url']
df['ner'] = entities

df.to_json(path_ner_data, orient='records', force_ascii=False, lines=True)

df.head()

,url,ner
0,https://news.tut.by/550306.html,"[(Шахтера, ORG), (Олег Кубарев, PER), (Шахтер,..."
1,https://news.tut.by/550307.html,"[(Илья Дударев, PER), (ЖРЭУ, ORG), (Брестская ..."
2,https://news.tut.by/550308.html,"[(МЧС, ORG), (Парке культуры и отдыха, ORG), (..."
3,https://news.tut.by/550309.html,"[(БАТЭ, ORG), (Борисов-Арены, ORG), (Жальгирис..."
4,https://news.tut.by/550310.html,"[(Генпрокуратура, ORG), (Украины, LOC), (Викто..."
